In [5]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

def reshape_sentinel2_to_sentinel1(sentinel1_path, sentinel2_path, output_path):
    # Open Sentinel-1 image to get its profile and transform
    with rasterio.open(sentinel1_path) as sentinel1:
        sentinel1_profile = sentinel1.profile
        sentinel1_transform = sentinel1.transform

    # Open Sentinel-2 image to get its data
    with rasterio.open(sentinel2_path) as sentinel2:
        sentinel2_profile = sentinel2.profile
        sentinel2_data = sentinel2.read()

    # Calculate the transform and dimensions for the output image
    transform, width, height = calculate_default_transform(
        sentinel2.crs, sentinel1.crs, sentinel1.width, sentinel1.height, *sentinel1.bounds)
    
    # Update the profile for Sentinel-2 to match Sentinel-1
    sentinel2_profile.update({
        'crs': sentinel1.crs,
        'transform': sentinel1_transform,
        'width': sentinel1.width,
        'height': sentinel1.height
    })
    
    # Create an empty array to store resampled data
    resampled_data = np.empty(shape=(sentinel2_data.shape[0], height, width), dtype=sentinel2_data.dtype)
    
    # Resample each band
    for i in range(sentinel2_data.shape[0]):
        reproject(
            source=sentinel2_data[i],
            destination=resampled_data[i],
            src_transform=sentinel2.transform,
            src_crs=sentinel2.crs,
            dst_transform=sentinel1_transform,
            dst_crs=sentinel1.crs,
            resampling=Resampling.nearest
        )
    
    # Save the resampled Sentinel-2 image
    with rasterio.open(output_path, 'w', **sentinel2_profile) as dst:
        dst.write(resampled_data)

# Example usage
sentinel1_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1\resampled_S1A_IW_SLC__1SDV_20230101T005408_geo_10m.tif"
sentinel2_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2\subset_0_of_S2A_MSIL2A_20230310T052641_N0509_R105_T43QEE_20230310T091359_resampled.tif"
output_path = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2\resampled_subset_0_of_S2A_MSIL2A_20230310T052641_N0509_R105_T43QEE_20230310T091359_resampled.tif"

reshape_sentinel2_to_sentinel1(sentinel1_path, sentinel2_path, output_path)
